In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [17]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [18]:
idx = 0

In [26]:
chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [ ]:
cur_month = 3
cur_year = 1986
target_year = 2009

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=crude+oil&tbs=bkt:s,cdr:1,cd_min:{cur_month}/1/{cur_year},cd_max:{cur_month}/31/{cur_year}&tbm=bks"
    driver.get(url)
    
    time.sleep(30)
    
    while True:
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso>div')
        if not news_results: 
            break  
            
        for news_div in news_results:
            try:
                Link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                title_element = news_div.find_element(By.CSS_SELECTOR, 'a>h3')
                Title = title_element.text.strip(".")
                Title = re.sub(r"- \d+페이지", "", Title)
                news_elements = news_div.find_elements(By.CSS_SELECTOR, 'div')
                Date = ''  # Default Date value if not found
                if len(news_elements) >= 10:
                    info = news_elements[10].text  # Assuming 'info' is in the 11th element
                    info_text = info.split("·")
                    if len(info_text) > 1:  # Check if Date is available
                        Date = info_text[1].strip()
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1

            except Exception as e:
                print(e)
            
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(2)
        except NoSuchElementException:
            break

    if cur_month == 12:  
        cur_month = 1
        cur_year += 1
    else: 
        cur_month += 1

In [22]:
new_df

,title,url,date
0,Using Microbes To Increase Recovery Of Oil. By...,https://books.google.co.kr/books?id=KgROAAAAIB...,1986. 1. 14.
1,Speculation Follows Drop In Oil Prices.,https://books.google.co.kr/books?id=WbdjAAAAIB...,1986. 1. 21.
2,Oil Price Decline Could Continue. By Kathleen ...,https://books.google.co.kr/books?id=6J4_AAAAIB...,1986. 1. 21.
3,Iran To Cut Crude Oil Output In Half In Respon...,https://books.google.co.kr/books?id=WklPAAAAIB...,1986. 1. 27.
4,Iran To Cut Crude Oil Output In Half In Respon...,https://books.google.co.kr/books?id=WUlPAAAAIB...,1986. 1. 26.
...,...,...,...
395,Robert J. Samuelson Oil Excise Tax Best For Th...,https://books.google.co.kr/books?id=I-lVAAAAIB...,1986. 2. 21.
396,Gas Cost Fueling Inquiries.,https://books.google.co.kr/books?id=C30_AAAAIB...,1986. 2. 7.
397,Incorporate Road Tax Into Cost Of Petrol.,https://books.google.co.kr/books?id=0_kPAAAAIB...,1986. 2. 27.
398,First Of A Two-Part Series. Oil Price Decline ...,https://books.google.co.kr/books?id=hJdeAAAAIB...,1986. 2. 9.
